In [7]:
import random
from abc import ABC, abstractmethod

# Classe para representar uma rainha
class Rainha:
    def __init__(self, linha, coluna):
        self.linha = linha
        self.coluna = coluna

    def __eq__(self, other):
        return self.linha == other.linha and self.coluna == other.coluna

    def __str__(self):
        return f"Rainha(linha={self.linha}, coluna={self.coluna})"

# Classe para representar o tabuleiro
class Tabuleiro:
    def __init__(self, linhas, colunas, rainhas=None):
        self.linhas = linhas
        self.colunas = colunas
        self.rainhas = rainhas if rainhas else [Rainha(random.randint(0, linhas - 1), col) for col in range(colunas)]
    
    def start(self):
        self.rainhas = [Rainha(random.randint(0, self.linhas - 1), col) for col in range(self.colunas)]
    
    def adicionar_rainha(self, rainha):
        self.rainhas.append(rainha)
    
    def remover_rainha(self, rainha):
        self.rainhas.remove(rainha)
    
    def calcular_conflitos(self):
        conflitos = 0
        for i in range(self.colunas):
            for j in range(i + 1, self.colunas):
                r1, r2 = self.rainhas[i], self.rainhas[j]
                if r1.linha == r2.linha or abs(r1.linha - r2.linha) == abs(r1.coluna - r2.coluna):
                    conflitos += 1
        return conflitos
    
    def clonar(self):
        return Tabuleiro(self.linhas, self.colunas, [Rainha(r.linha, r.coluna) for r in self.rainhas])
    
    def __str__(self):
        return "\n".join(str(rainha) for rainha in self.rainhas)

# Interface para algoritmos de resolução
class AlgoritmoResolucao(ABC):
    @abstractmethod
    def resolver(self, tabuleiro):
        pass

# Implementação do Algoritmo Genético
class AlgoritmoGenetico(AlgoritmoResolucao):
    def __init__(self, tamanho_populacao=100, taxa_mutacao=0.1, geracoes=1000):
        self.tamanho_populacao = tamanho_populacao
        self.taxa_mutacao = taxa_mutacao
        self.geracoes = geracoes
    
    def resolver(self, tabuleiro):
        populacao = [tabuleiro] + [Tabuleiro(tabuleiro.linhas, tabuleiro.colunas) for _ in range(self.tamanho_populacao - 1)]
        
        for _ in range(self.geracoes):
            populacao = sorted(populacao, key=lambda t: t.calcular_conflitos())
            if populacao[0].calcular_conflitos() == 0:
                return populacao[0]
            
            nova_populacao = populacao[:10]  # Elitismo: mantém os melhores
            while len(nova_populacao) < self.tamanho_populacao:
                pai, mae = random.sample(populacao[:50], 2)
                filho = self._crossover(pai, mae)
                if random.random() < self.taxa_mutacao:
                    self._mutacao(filho)
                nova_populacao.append(filho)
            
            populacao = nova_populacao
        
        return sorted(populacao, key=lambda t: t.calcular_conflitos())[0]
    
    def _crossover(self, pai, mae):
        ponto_corte = random.randint(0, pai.colunas - 1)
        novas_rainhas = pai.rainhas[:ponto_corte] + mae.rainhas[ponto_corte:]
        return Tabuleiro(pai.linhas, pai.colunas, novas_rainhas)
    
    def _mutacao(self, tabuleiro):
        coluna = random.randint(0, tabuleiro.colunas - 1)
        tabuleiro.rainhas[coluna] = Rainha(random.randint(0, tabuleiro.linhas - 1), coluna)

# Classe para visualizar o tabuleiro
class VisualizadorTabuleiro:
    @staticmethod
    def mostrar(tabuleiro):
        print("Conflitos: ", tabuleiro.calcular_conflitos())
        print("Solução encontrada:")
        for linha in range(tabuleiro.linhas):
            print(" ".join("Q" if any(r.linha == linha and r.coluna == coluna for r in tabuleiro.rainhas) else "." for coluna in range(tabuleiro.colunas)))


In [13]:
# Inicializa o tabuleiro com 8 rainhas em posições aleatórias
tabuleiro = Tabuleiro(8, 8)
tabuleiro.start()

algoritmo = AlgoritmoGenetico(tamanho_populacao=100, geracoes=1000)
solucao = algoritmo.resolver(tabuleiro)

VisualizadorTabuleiro.mostrar(solucao)

Conflitos:  0
Solução encontrada:
. . Q . . . . .
. . . . . Q . .
. . . . . . . Q
Q . . . . . . .
. . . Q . . . .
. . . . . . Q .
. . . . Q . . .
. Q . . . . . .


In [14]:
# Inicializa o tabuleiro com 15 rainhas em posições aleatórias
tabuleiro = Tabuleiro(15, 15)
tabuleiro.start()

algoritmo = AlgoritmoGenetico(tamanho_populacao=100, geracoes=1000)
solucao = algoritmo.resolver(tabuleiro)

VisualizadorTabuleiro.mostrar(solucao)

Conflitos:  1
Solução encontrada:
. . . . . . . . . Q . . . . .
. Q . . . . . . . . . . . . .
. . . . Q . . . . . . . . . .
. . . . . . . Q . . . . . . .
Q . . . . . . . . . . . . . .
. . . . . . . . . . Q . . . .
. . . . . . . . . . . . Q . .
. . . . . . . . . . . . . . Q
. . . Q . . . . . . . . . . .
. . . . . . Q . . . . . . . .
. . . . . . . . . . . . . Q .
. . Q . . . . . . . . . . . .
. . . . . Q . . . . . . . . .
. . . . . . . . . . . Q . . .
. . . . . . . . Q . . . . . .
